# 6 — Camera Setup in ROS2

> Connect a USB/CSI camera, publish image topics, and visualize with RViz2.

**Goal:** Get a camera streaming images into ROS2 topics and understand the image pipeline.

In [ ]:
#| default_exp camera_setup

## 6.1 How Cameras Work in ROS2

The camera pipeline in ROS2:

```
Physical Camera  →  Driver Node  →  ROS Topics  →  Your Processing Node
  (USB/CSI)       (usb_cam /       /camera/image_raw    (subscriber)
                   v4l2_camera)    /camera/camera_info
```

### Topics Published by Camera Drivers

| Topic | Message Type | Content |
|-------|-------------|--------|
| `/camera/image_raw` | `sensor_msgs/msg/Image` | Raw pixel data (BGR8 or RGB8) |
| `/camera/camera_info` | `sensor_msgs/msg/CameraInfo` | Calibration data (intrinsic matrix, distortion coefficients) |
| `/camera/image_raw/compressed` | `sensor_msgs/msg/CompressedImage` | JPEG compressed image (via image_transport) |

### Why Two Topics?

- **`image_raw`**: Full uncompressed pixel data — needed for accurate CV processing
- **`camera_info`**: Lens calibration — needed for undistortion, 3D projection, stereo vision
- **`compressed`**: Bandwidth-efficient version for remote viewing / recording

## 6.2 Verify Your Camera (Before ROS2)

Before starting the ROS2 driver, make sure Linux sees your camera:

```bash
# Check the device exists
ls /dev/video*
# Expected: /dev/video0 /dev/video1

# Get camera details
sudo apt install -y v4l-utils
v4l2-ctl --list-devices

# List supported formats and resolutions
v4l2-ctl --device=/dev/video0 --list-formats-ext

# Quick test — capture and display (if you have a GUI)
sudo apt install -y ffmpeg
ffplay /dev/video0
```

### WSL2 USB Passthrough Reminder

If on WSL2, you need `usbipd` (covered in notebook 01):

```powershell
# PowerShell (Admin)
usbipd list                          # find your camera's BUSID
usbipd bind --busid <BUSID>
usbipd attach --wsl --busid <BUSID>
```

## 6.3 Option A: `usb_cam` — The Classic USB Camera Driver

**usb_cam** is the most widely used USB camera driver in ROS2.

### Run Directly

```bash
# Simple start with defaults
ros2 run usb_cam usb_cam_node_exe

# With parameters
ros2 run usb_cam usb_cam_node_exe --ros-args \
  -p video_device:=/dev/video0 \
  -p image_width:=640 \
  -p image_height:=480 \
  -p framerate:=30.0 \
  -p pixel_format:=yuyv
```

### Important Parameters

| Parameter | Default | Description |
|-----------|---------|------------|
| `video_device` | `/dev/video0` | Camera device path |
| `image_width` | 640 | Image width in pixels |
| `image_height` | 480 | Image height in pixels |
| `framerate` | 30.0 | Frames per second |
| `pixel_format` | `yuyv` | Camera pixel format (`yuyv`, `mjpeg`) |
| `camera_frame_id` | `camera` | TF frame name |
| `io_method` | `mmap` | I/O method (`mmap`, `read`, `userptr`) |

## 6.4 Option B: `v4l2_camera` — Lightweight Alternative

**v4l2_camera** is a simpler, more modern driver:

```bash
ros2 run v4l2_camera v4l2_camera_node --ros-args \
  -p video_device:=/dev/video0 \
  -p image_size:=[640,480] \
  -p output_encoding:=rgb8
```

### Which Driver to Choose?

| Feature | usb_cam | v4l2_camera |
|---------|---------|-------------|
| Maturity | Very mature, widely used | Newer, simpler |
| Configuration | More parameters | Fewer, simpler |
| Compressed output | Built-in | Via image_transport plugin |
| Recommendation | Start here | Try if usb_cam has issues |

> **Use `usb_cam` unless you hit problems.** It has more community support.

## 6.5 Verify the Camera Stream

Once the driver is running, verify it from another terminal:

```bash
# List topics — look for image topics
ros2 topic list
# /camera/image_raw
# /camera/camera_info
# /camera/image_raw/compressed

# Check publish rate
ros2 topic hz /image_raw
# average rate: 30.000

# Check bandwidth
ros2 topic bw /image_raw
# ~27.6 MB/s for 640x480 BGR8 @ 30fps

# See message details
ros2 topic info /image_raw --verbose
```

## 6.6 Visualize with RViz2

RViz2 is the standard visualization tool. To see your camera:

```bash
# Launch RViz2
rviz2
```

Then in the RViz2 GUI:

1. Click **Add** (bottom left)
2. Select **By topic** tab
3. Expand `/camera/image_raw` → select **Image**
4. Click **OK**

You should see your live camera feed in RViz2!

### Alternative: `rqt_image_view`

For a simpler image viewer:

```bash
ros2 run rqt_image_view rqt_image_view
```

Select your topic from the dropdown — instant camera preview.

## 6.7 Launch File for the Camera

For reproducible startup, create a launch file:

In [ ]:
#| export

CAMERA_LAUNCH = '''
#!/usr/bin/env python3
"""Launch file for the USB camera."""

from launch import LaunchDescription
from launch.actions import DeclareLaunchArgument
from launch.substitutions import LaunchConfiguration
from launch_ros.actions import Node


def generate_launch_description():
    # Declare launch arguments (overridable from command line)
    video_device_arg = DeclareLaunchArgument(
        'video_device', default_value='/dev/video0',
        description='Path to the video device'
    )
    framerate_arg = DeclareLaunchArgument(
        'framerate', default_value='30.0',
        description='Camera framerate'
    )

    # Camera driver node
    camera_node = Node(
        package='usb_cam',
        executable='usb_cam_node_exe',
        name='camera',
        namespace='camera',
        parameters=[{
            'video_device': LaunchConfiguration('video_device'),
            'image_width': 640,
            'image_height': 480,
            'framerate': LaunchConfiguration('framerate'),
            'pixel_format': 'yuyv',
            'camera_frame_id': 'camera_optical_frame',
        }],
        # Remap to standard names
        remappings=[
            ('image_raw', 'image_raw'),
        ]
    )

    return LaunchDescription([
        video_device_arg,
        framerate_arg,
        camera_node,
    ])
'''

print(CAMERA_LAUNCH)

### Using the Launch File

Save the above as `~/ros2_ws/src/robot_vision/launch/camera_launch.py`, then:

```bash
# Build and source
cd ~/ros2_ws && colcon build --symlink-install && source install/setup.bash

# Launch with defaults
ros2 launch robot_vision camera_launch.py

# Launch with overrides
ros2 launch robot_vision camera_launch.py video_device:=/dev/video2 framerate:=15.0
```

## 6.8 Understanding Image Transport

Raw images are **huge** (640×480×3 bytes = ~900 KB per frame). At 30 fps that's ~27 MB/s!

**image_transport** provides transparent compression:

| Transport | Topic Suffix | Size | Use Case |
|-----------|-------------|------|----------|
| `raw` | (none) | ~900 KB | Local processing |
| `compressed` | `/compressed` | ~30-50 KB | Remote viewing, logging |
| `theora` | `/theora` | ~10-20 KB | Video streaming |

image_transport works automatically — the camera driver publishes raw, and the transport plugins create compressed versions.

```bash
# See available transports
ros2 run image_transport list_transports

# Subscribe to compressed images
ros2 topic echo /camera/image_raw/compressed
```

### Subscribing with image_transport in Python

```python
# Instead of subscribing to the raw topic directly,
# you can use image_transport for automatic decompression:
# (But for simplicity, we'll subscribe to raw in this course)
self.create_subscription(Image, '/camera/image_raw', self.callback, 10)
```

## 6.9 Camera Calibration (Overview)

Real lenses have **distortion** (barrel, pincushion). For accurate measurements, you need to **calibrate** the camera.

```bash
# Print a checkerboard pattern (8x6) and run:
ros2 run camera_calibration cameracalibrator \
  --size 8x6 \
  --square 0.025 \
  --ros-args -r image:=/camera/image_raw
```

The calibration tool:
1. Asks you to show the checkerboard from different angles
2. Computes the intrinsic matrix and distortion coefficients
3. Saves to a YAML file that the camera driver loads

After calibration, `/camera/camera_info` contains accurate data, and `image_proc` can undistort images.

> **For this course, we skip calibration** — it's only needed for precise measurements.

## 6.10 A Minimal Python Camera Subscriber

Here's how to receive images in your own node (preview for notebook 07):

In [ ]:
#| export

CAMERA_SUBSCRIBER = '''
#!/usr/bin/env python3
"""Subscribes to camera images and logs frame info."""

import rclpy
from rclpy.node import Node
from rclpy.qos import qos_profile_sensor_data
from sensor_msgs.msg import Image


class CameraSubscriber(Node):
    def __init__(self):
        super().__init__('camera_subscriber')

        # Subscribe with sensor QoS (BEST_EFFORT — matches camera driver)
        self.subscription = self.create_subscription(
            Image,
            '/camera/image_raw',
            self.image_callback,
            qos_profile_sensor_data     # <-- important!
        )
        self.frame_count = 0
        self.get_logger().info('Camera subscriber ready')

    def image_callback(self, msg: Image):
        self.frame_count += 1
        if self.frame_count % 30 == 0:  # Log every 30 frames
            self.get_logger().info(
                f'Frame #{self.frame_count}: '
                f'{msg.width}x{msg.height} '
                f'encoding={msg.encoding} '
                f'data_size={len(msg.data)} bytes'
            )


def main(args=None):
    rclpy.init(args=args)
    node = CameraSubscriber()
    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        pass
    finally:
        node.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()
'''

print(CAMERA_SUBSCRIBER)

### Key Detail: QoS Matching

Camera drivers publish with **`BEST_EFFORT`** QoS. If your subscriber uses `RELIABLE` (the default when you pass just an integer), the connection will **silently fail**.

Always use `qos_profile_sensor_data` when subscribing to camera topics.

## 6.11 Troubleshooting Camera Issues

| Problem | Cause | Solution |
|---------|-------|----------|
| No `/dev/video0` | Camera not detected (WSL2: not passed through) | Check `usbipd attach` |
| Driver starts but no data | Wrong pixel format | Try `pixel_format:=mjpeg` |
| Subscriber gets no data | QoS mismatch | Use `qos_profile_sensor_data` |
| Low frame rate | Resolution too high / USB bandwidth | Lower resolution or framerate |
| Permission denied | User not in `video` group | `sudo usermod -aG video $USER` then relog |
| Image is green/garbled | Wrong encoding | Check `v4l2-ctl --list-formats-ext` |

### Debugging Commands

```bash
# Check if camera works outside ROS2
ffplay /dev/video0

# Check topic QoS
ros2 topic info /camera/image_raw --verbose

# Check if data arrives (shows first message)
ros2 topic echo /camera/image_raw --once
```

---

**Next →** [Notebook 07: Image Processing with OpenCV](07_opencv_processing.ipynb)